<a href="https://colab.research.google.com/github/GuillBla/Internship_Report_Code/blob/main/Simple_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A script to execute to prevent from the limits of imposed by Google Colab

In [ ]:
# function ConnectButton(){
#     console.log("Connect pushed"); 
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
# }
# setInterval(ConnectButton,60000);

Install packages on Colab

In [ ]:
!pip install split-folders
# !pip install keras.utils.plot_model  
# Uncomment the lines associated with plot_model and matplotlib.pyplot if you want a visual description of the model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
import zipfile
import os
import shutil
import splitfolders
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
# import matplotlib.pyplot as plt
# from tensorflow.keras.utils import plot_model

In [ ]:
drive.mount('/content/drive')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-3b8a479202a4>", line 1, in <module>
    drive.mount('/content/drive')
  File "/usr/local/lib/python3.7/dist-packages/google/colab/drive.py", line 105, in mount
    ephemeral=True)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/drive.py", line 120, in _mount
    'request_auth', request={'authType': 'dfs_ephemeral'}, timeout_sec=None)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_message.py", line 171, in blocking_request
    return read_reply_from_input(request_id, timeout_sec)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_message.py", line 97, in read_reply_from_input
    time.sleep(0.025)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local

KeyboardInterrupt: ignored

If the following folders have already been created, they need to be removed 

In [ ]:
# # Clear the environment
# shutil.rmtree(r'/content/topic_classification_splitted')
# shutil.rmtree(r'/content/topic_classification')

In [ ]:
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/topic_classification.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/content') #Extracts the files into the /tmp folder
zip_ref.close()

I split the dataset into two folders : 
1. One containing the training set : /content/topic_classification_splitted/train
2. One containing the testing set : /content/topic_classification_splitted/test

Note that I will split the first folder in two to create a validation set.


In [ ]:
splitfolders.ratio(r"/content/topic_classification",
                   output=r"/content/topic_classification_splitted",
                   seed=2504, ratio=(.8, .2), group_prefix=None, move=False)

os.rename("/content/topic_classification_splitted/val", "/content/topic_classification_splitted/test") 

I implement data augmentation on the training data and I split the training dataset in two using the radio val_ratio

In [ ]:
BATCH_SIZE = 64
download_dir = Path(r"/content/topic_classification_splitted")
train_data_dir = download_dir/'train'
test_data_dir = download_dir/'test'
class_subset = sorted(os.listdir(train_data_dir))
val_ratio = 0.15
target_size = (224, 224)

train_generator = ImageDataGenerator(
                                    #  rotation_range=90, 
                                    #  brightness_range=[0.1, 0.7],
                                    #  width_shift_range=0.5, 
                                    #  height_shift_range=0.5,
                                    #  horizontal_flip=True, 
                                    #  vertical_flip=True,
                                     validation_split=val_ratio,
                                     preprocessing_function=preprocess_input) # Xception preprocessing

traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=target_size,
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=target_size,
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # Xception preprocessing

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=target_size,
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

A function to build the architecture of the model. It takes in input : 
1. The size of the images (it must match the target_size used previously
2. The number of categories in the dataset, it is also the size of the output layer
3. The optimizer instantiated for training (default 'RMSProp')
4. The number of layers from the pre-trained model to unfreeze. If set to 0, all of the pre-trained will freeze

In [ ]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):

    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = Resnet50(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Add layers on top of the model
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Plot the model
    # plot_model(model, to_file='model.png',show_layer_activations=True)

    # Display the image
    # data = plt.imread('model.png')
    # plt.imshow(data)
    # plt.show()

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [ ]:
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=27

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 10

# First we'll train the model without Fine-tuning
model = create_model(input_shape, n_classes, optim_1, fine_tune=0)

In [ ]:
tl_checkpoint_1 = ModelCheckpoint(filepath='model.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

In [ ]:
history = model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1, early_stop],
                            verbose=1)

In [ ]:
# Generate predictions
model.load_weights('model.weights.best.hdf5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

preds = model.predict(testgen)
pred_classes = np.argmax(preds, axis=1)

In [ ]:
# The score results :
accuracy = accuracy_score(true_classes, pred_classes)
print("Model Accuracy without Fine-Tuning: {:.2f}%".format(accuracy * 100))
p = precision_score(true_classes, pred_classes, average='macro')
r = recall_score(true_classes, pred_classes, average='macro')
F1 = 2 * (p * r) / (p + r)
print("Precision = " + str(p))
print("Recall = " + str(r))
print("F1 score = " + str(F1))